<a href="https://colab.research.google.com/github/tonytani37/stats/blob/main/%E3%82%B9%E3%82%BF%E3%83%83%E3%83%84%E3%83%87%E3%83%BC%E3%82%BF%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# まずは前提条件
Googleアカウントが必須です。

ブラウザは最新のChromeの利用を強く推奨します。(Microsoft Edgeの最新版は大丈夫ですが、旧EdgeやIEは動かないかもしれません。FirefoxやSafariでは動作を確認しました。確認したブラウザはMacOS版です）

## 注意事項
### 処理自体は膨大な能力を持つGoogleクラウド上で実行されるので、目の前のパソコンが多少古くてもあまり心配することはありません。

１．説明を読んで、上から順番に番号順に処理をしてください。

２．処理によっては実行するとその下によくわからない文字がずらずらと表示されるものとされないものがあります。どのタイミングで処理が終わったかわかりづらいかもしれないので、各処理が完了すると「x おわり」と表示するようにしていますで、そこまで我慢してください。（表示されるメッセージのnは処理の番号です）

３．処理を実行したあと、左の丸に三角印が赤くなるとエラーが発生しています。その場合は、もう一度最初から実行してみましょう。それでもエラーが解消しない場合には vka77606@gmail.com までメッセージを下さい。




# さあ、はじめましょう
ここから処理が始まります。

### 最初にデータ化したいPDFファイルを入手します
1-1 は自分のPCにあるPDFファイルを変換したい場合に使います。

1-2 はよくわからないけどとりあえず練習用に試してみたい人向けに、Xリーグのホームページにある2019/8/24の富士通フロンティアーズ対IBM BigBlueの対戦のものがダウンロードされます。

どちらか一つ、好きな方を実行してください。

# 初回の実行時に「これはGoogleが提供してないけど実行する？」的なメッセージが出るかも知れませんが、気にせず右側のボタン（薄い表示く表示されてます）をクリックして実行して下さい。

### PDFファイルの準備 その１
自分のPCにあるPDFファイルを変換したい場合
注意）ファイルは/content/直下にアップロードされますので、変換処理前に/content/pdf/に移動させてください

ファイルサイズが大きいとうまくアップロードできない場合がありました。その場合には直接アップロードしてください。

In [ ]:
# 1-1
#OCRであつかうPDFファイルを読み込む（変換したいPC上のファイルを指定してくだされ）
from google.colab import files
!rm -r /content/pdf/
!mkdir /content/pdf/

f = files.upload()
#読み込んだ画像のファイル名
filename = list(f)[0]

print('1-1 おわり')

### PDFファイルの準備 その２）

とりあえず試してみたい場合はこちら。ダウンロードされたPDFファイルは/content/pdf/にダウンロードされます。　

In [ ]:
# 1-2
#OCRであつかうPDFファイルを読み込む（練習用に特定のPDFファイルをダウンロードしてくる）
!rm -r /content/pdf/
!mkdir /content/pdf/

! wget -c -P '/content/pdf/' 'https://xleague.s3.ap-northeast-1.amazonaws.com/wp-content/uploads/2019/07/20190824_X1_FF_BB_921.pdf'
# filename = "20190824_X1_FF_BB_921.pdf"

print('1-2 おわり')

### データ変換のための準備を行います

In [ ]:
# 2
# PDFから画像へ変換するためのライブラリ、ツールとOCR読み込みに必要なツールをインストールします
!apt-get install poppler-utils 
!pip install pdf2image

!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pyocr

#日本語トレーニングデータのダウンロードとして環境設定します
!curl -L https://github.com/tesseract-ocr/tessdata/raw/master/jpn.traineddata > jpn.traineddata
!cp jpn.traineddata /usr/share/tesseract-ocr/4.00/tessdata

# PythonでOCR読み込み変換に必要なライブラリを準備します
from PIL import Image
import pyocr
import pyocr.builders

print('4 おわり')

### 画像データへ変換する処理を行います

In [ ]:
# 3
# 画像データに変換する処理
from pathlib import Path
from pdf2image import convert_from_path
import glob

# Google Colabのデフォルト環境内にimageというフォルダを作る
!rm -r /content/image/
!mkdir /content/image/

def pdf2img(filename):
    pdf_path = Path(filename)
    #outputのファイルパス（この場合は上でつくったimageフォルダを指定）
    img_path=Path("/content/image/")

    #変換されたjpegファイルがimg_pathで指定されたフォルダに保存されます
    convert_from_path(pdf_path, output_folder=img_path,fmt='jpeg',output_file=pdf_path.stem)

#フォルダにあるPDFファイルの一覧を取得する
pdf_list = glob.glob('/content/pdf/*.pdf')

for row in pdf_list:
    pdf2img(row)

print('3 おわり')

3 おわり


### 以上で画像変換完了。
/content/image/フォルダにたくさんの画像ファイル(...jpg)が作られているはずです。

### ここからOCRからデータ抽出処理です。

ひとつのPDFファイルで５分くらいかかる場合があります。

左側の丸ボタン（中に白三角）の周りがぐるぐる回らなくなり、「4 おわり」が表示されるまで我慢強く待ちましょう。

In [ ]:
# 4
#読み込んだ画像をOCRでテキスト抽出。

# OCR処理のためのインスタンス作成
tools = pyocr.get_available_tools()
tool = tools[0]
langs = tool.get_available_languages()

#うまく変換できなかった文字を強制的に置き換えるための文字候補一覧
conv_text = {"①":"1","②":"2","③":"3","④":"4","⑤":"5","⑥":"6","⑦":"7","⑧":"8","⑨":"9",
             "⑩":"10","⑪":"11","⑫":"12","⑬":"13","⑭":"14","⑮":"15",
             "⑯":"16","⑰":"17","⑱":"18","⑲":"19","⑳":"20",',':'',"&":"","-":"",
             "PASS":"PASS","RUN":"RUN","FG":"FG","PUNT":"PUNT","し":"L","、":" "}

#テキストを保存するフォルダの作成
!rm -r /content/ocr/
!mkdir /content/ocr/


def ocr2txt(fname):
    txt = tool.image_to_string(
        Image.open(fname),
        lang="jpn",
        builder=pyocr.builders.TextBuilder(tesseract_layout=6)
        )
    # プレーに関係ありそうな文字として　"&",'Penalty','Kick-off','Extra Point','TIMEOUT','Quarter'が含まれる行だけ抽出する
    # そのほかの特定文字も抽出対象としたい場合は ifの次に or ('' & in line) を追加挿入し、''の間に含めたい文字列を指定するとできますよ
    return  [line for line in txt.splitlines() if ('&' in line) or ('Penalty' in line) or ('Kick-off' in line) or ('Extra Point' in line) or ('TIMEOUT' in line) or (':' in line) or ('Quarter' in line)]
    
def num_conv(line):
    for ct in conv_text:
        line = line.replace(ct,conv_text[ct])
    return line


#フォルダに出力された画像ファイルの一覧を取得
jpg_list = glob.glob('/content/image/*.jpg')

#画像ファイル群をOCRでデータ化し、それを一つのファイルにまとめる
output_file = []
for row in sorted(jpg_list):
    output_file.append([row1.replace(' ','') for row1 in [num_conv(out_txt) for out_txt in ocr2txt(row)]])

# List(output_file)から読みだしたデータに一行づつ改行コードを追加して出力用ファイルの準備をする
output_txt = ''
for x in output_file:
    for z in x:
        output_txt += z+'\n'


#最後にテキストファイルを（とりあえず）出力してなくさないようにする
output_file_txt = '/content/ocr/'+jpg_list[0][15:-4]
with open(output_file_txt,mode='w') as f:
    f.write(output_txt)

print('4 おわり')

4 おわり


### OCRで読み込んだデータをCSV形式に変換します

前の処理と比べてこちらはすぐに終わります。
（何度処理をしても大丈夫です）

In [78]:
# 5
import re
import glob
import pandas as pd 

# 読み込むチームデータを設定します。{'チーム名の先頭から５文字':'チームの略称'}
team_name ={'富士通フロ':'FF'
                ,'IBMBi':'BB'
                ,'パナソニッ':'PI'
                ,'エレコム神':'EF'
                ,'オービック':'OS'
                ,'東京ガスク':'TG'
                ,'オール三菱':'OM'
                ,'ノジマ相模':'NR'
                ,
            }

tname = [row for row in team_name.values()]

filename = glob.glob('/content/ocr/*')[0]

with open(filename,mode='r',encoding='UTF-8') as f:
    file_in = f.readlines()

in_list = [row for row in file_in]

class data_conv():
    def __init__(self):
        self.team = 'XX'
        self.qt = 0

    def find(self,s):
        fp = re.search(r'\w\w\d+[LMR]', s) # 'チーム名','フィールドポジション','ハッシュ'の組み合わせを探している
        if fp: # 処理対象の行に上の組み合わせが存在する場合
            fp_all = fp.group()
            fieldpos = fp_all[:2]
            y = re.search(r'\d+',fp_all[2:])
            yardline = y.group()
            hash = fp_all[-1]
            # down
            down = s[:1]
            # togo 
            tg = re.search(r'\d+',s[1:])
            if tg is not None:
                togo = tg.group()
            else:
                togo = 0 #データが見つからなかったら0

            # ヤード数
            yd = re.search(r'-*\d+y',s)
            if yd is not None:
                if 'ロス' in s:
                    yards = 0 - int(yd.group()[:-1]) # ロスという文字があったらマイナスゲイン
                else:
                    yards = yd.group()[:-1]
            else:
                yards = 0 # データが見つからなかったら0

            # プレーの種類を設定
            if 'RUN' in s:
                play = 'RUN'
            elif 'PASS' in s:
                play = 'PASS'
            elif 'PUNT' in s:
                play = 'PUNT'
            elif 'FG' in s:
                play = 'FG'
            else:
                play = 'None' # 見つからなかったらNone

            # オフェンスチームから見て敵陣に入ったときのフィールドポジションの設定
            if team != fieldpos:
                yardline = 100- int(yardline)

            # 後で検証できるように最後に(other)データを全部くっつけておく（データ分析には不要）
            other = s.replace(',','')
            other = other.replace('\n','')
            return self.qt,self.team,int(down),int(togo),fieldpos,int(yardline),hash,play,int(yards),other
        else:
            return s[:-1],0,0,0,0,0,0,0

    def set_q(self,s):
        quarter = re.search(r'Quarter', s)
        if quarter:
            fq,sq,tq,yd = '','','',''
            fq = re.search(r'First', s)
            sq = re.search(r'Second', s)        
            tq = re.search(r'Third', s)        
            yq = re.search(r'Fourth', s)
            if fq:
                q = 1
            elif sq:
                q = 2
            elif tq:
                q = 3
            elif yq:
                q = 4
            self.qt = q

    def teams(self,s):
        for row in team_name:
            if row in s:
                self.team = team_name[row]

lines_strip_output = []
# team_set = set()

# クォーター、チーム名がうまくOCR変換されなかった場合,クォーターは0、チーム名はXXを初期値とする
# クォーターが0だったり、クォーターが存在しない場合、チーム名にXXのものがある場合は、OCR変換がうまく言ってないので手修正が必要ですよ

d = data_conv() # データ変換用クラスからインスタンスを作る

# team_set.add(t)
for s in in_list:
    d.set_q(s)
    d.teams(s[:5])
    lines_strip_output.append(d.find(s))

output_csv = pd.DataFrame(lines_strip_output,columns=['quarter','offence','down','togo','side','yardline','hash','play','yards','other'])
# output_csv[(output_csv['offence']==list(team_set)[0])|(output_csv['offence']==list(team_set)[1])].to_csv(filename[14:]+'.csv',index=False)

# チーム名が入っている（0以外）のデータを抜いてCSVファイルにしていますが、もしかするとうまくデータが抜けないこともあるかもしれません。
output_csv[-(output_csv['offence']== 0)].to_csv(filename[14:]+'.csv',index=False)

print('5 おわり')

5 おわり


データの抽出条件を一つづつ確認しながらコードを書いていたので、危うい部分や無駄な部分があることは自覚しています。またどんなOCR読み込み処理でどんなデータが出てくるか予測不可能な部分もあり、それらをすべて網羅した処理ではないと思われ、途中で異常終了することもありますが、とりあえず動くことを最優先にしてるので、そういった点についてはご容赦ください。（なんだか自衛隊の大規模接種会場予約システムみたいだな）